In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import IBMQ
from qiskit import Aer
from qiskit import execute
from qiskit.providers.aer.noise import NoiseModel
from qiskit.visualization import plot_histogram

In [ ]:
IBMQ.load_account()

In [ ]:
# use this function to plot measurement results
# alternatively, use plot_histogram
def visualize_counts(counts_dict, plt=plt):
    """
    take a dictionary of measurement results and
    plot it as a bar chart
    """
    N_bits = len(list(counts_dict.keys())[0])
    N_values = 2**N_bits
    
    values = [ i for i in range(N_values) ]
    labels = [ bin(i)[2:].zfill(N_bits) for i in range(N_values)]
    counts = [ counts_dict.get(label,0) for label in labels ]
    
    plt.bar(values, counts/np.sum(counts), tick_label=labels)
    plt.xlabel("measurement outcome")
    plt.ylabel("probability")
    return plt

In [ ]:
# define a quantum register, called 'q' with 2 qubits
# and a classical register, called 'c' with 1 bit
qr = QuantumRegister(2, 'q')
cr = ClassicalRegister(2, 'c')

# Exercise 1

In the demonstration, we only generated the Bell state $\beta_{00}$. Now, we want to generalize this to generate all four possible Bell states.

Write a function **prepare_Bell_state(x, qr, cr)** which takes as an input
* an integer number x between 0 and 4
* a two-qubit quantum register qr
* a two-bit classical register cr

and outputs a quantum circuit that generates the Bell state $\beta_{ij}$ where $ij$ denotes the binary encoding of $x$.

In other words, the output for **prepare_Bell_state(0,qr,cr)** should be
```text
      ░ ┌───┐     
q_0: ─░─┤ H ├──■──
      ░ └───┘┌─┴─┐
q_1: ─░──────┤ X ├
      ░      └───┘
c: 2/═════════════
```
the output for **prepare_Bell_state(1,qr,cr)** should be
```text
     ┌───┐ ░ ┌───┐     
q_0: ┤ X ├─░─┤ H ├──■──
     └───┘ ░ └───┘┌─┴─┐
q_1: ──────░──────┤ X ├
           ░      └───┘
c: 2/══════════════════
```
the output for **prepare_Bell_state(2,qr,cr)** should be
```text
           ░ ┌───┐     
q_0: ──────░─┤ H ├──■──
     ┌───┐ ░ └───┘┌─┴─┐
q_1: ┤ X ├─░──────┤ X ├
     └───┘ ░      └───┘
c: 2/══════════════════
```
and the output for **prepare_Bell_state(3,qr,cr)** should be
```text
     ┌───┐ ░ ┌───┐     
q_0: ┤ X ├─░─┤ H ├──■──
     ├───┤ ░ └───┘┌─┴─┐
q_1: ┤ X ├─░──────┤ X ├
     └───┘ ░      └───┘
c: 2/══════════════════
```
Hint: ░ is a barrier. 

In [ ]:
def prepare_Bell_state(x, qr, cr):
    # PUT YOUR CODE HERE

In [ ]:
# generate and visualize your quantum circuit:
qc = prepare_Bell_state(3, qr, cr)
qc.draw()

<hr>

# Exercise 2

In exercise 1, we constructed four different quantum circuits to generate the four Bell states. Perform a statevector simulation to calculate the final state vector of these four circuits. Convince yourself that these circuits are generating Bell states.

Note: You can calculate all state vectors in parallel if you group the corresponding circuits into a list.

In [ ]:
# PUT YOUR CODE HERE

<hr>

# Exercise 3a

Having checked that the four circuits generate Bell states, we now want to simulate their execution on a quantum computer. 

However, the only output a quantum computer can give to us are measurement results.
Therefore, take the four circuits and append measurement operations that determine the state of the qubits in the quantum register in the z basis. 

Do a QASM simulation of these circuits assuming an **ideal** quantum computer.
Visualize the measurement results and interpret them.

Hint: You can use the function visualize_counts() defined above to plot the result.

In [ ]:
# PUT YOUR CODE HERE

# Exercise 3b

Repeat the same steps assuming a **realistic noise model** taken from an existing quantum computer of your choice. 

In [ ]:
# PUT YOUR CODE HERE

<hr>

# Exercise 4

We would like to understand better the imperfections of a realistic quantum computer. 

To analyze the impact of different quantum gates on the errors, we want to add more gates to the quantum circuit without changing its output on an **ideal** quantum computer. A way to approach this task is to use that the Hadamard gate $H$ and the CNOT gate are their own inverses, i.e.,
$$ H \cdot H \cdot H = H $$
and
$$ CNOT \cdot CNOT \cdot CNOT = CNOT $$
That is, we can replace a single $H$ gate by an odd number of $H$ gates without (in theory) changing the output of the circuit. 
The same is true for CNOT gates. 
Of course, the output on a **noisy** quantum computer taking will change and this effect is precisely what we want to analyze. 

Define a modified function **prepare_Bell_state_gate_error_analysis(x, qr, cr, N_H, N_cnot)** which takes as an input
* an integer number x between 0 and 4
* a two-qubit quantum register qr
* a two-bit classical register cr
* an integer number N_H 
* an integer number N_cnot

and adds N_H pairs of Hadamard gates and N_cnot pairs of CNOT gates to the circuit. 
For instance, the output of **prepare_Bell_state_gate_error_analysis(1,qr,cr,1,1)** should be 
```text
     ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐      ░       ░       ░ ┌─┐   
q_0: ┤ X ├─░─┤ H ├─░─┤ H ├─░─┤ H ├──■───░───■───░───■───░─┤M├───
     └───┘ ░ └───┘ ░ └───┘ ░ └───┘┌─┴─┐ ░ ┌─┴─┐ ░ ┌─┴─┐ ░ └╥┘┌─┐
q_1: ──────░───────░───────░──────┤ X ├─░─┤ X ├─░─┤ X ├─░──╫─┤M├
           ░       ░       ░      └───┘ ░ └───┘ ░ └───┘ ░  ║ └╥┘
c: 2/══════════════════════════════════════════════════════╩══╩═
                                                           0  1 
```

Simulate circuits with different values of N_H and N_cnot with a realistic noise model for one fixed value of x. Visualize and interpret your results.

Note: Make sure to add barriers to the circuit to avoid cancellation of these gates in the transpiler step

In [ ]:
def prepare_Bell_state_gate_error_analysis(x, qr, cr, N_H, N_cnot):
    # PUT YOUR CODE HERE

<hr>